In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import cv2
import numpy as np
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras import backend as K
from PIL import ImageOps
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
print(f"TensorFlow использует GPU: {tf.config.list_physical_devices('GPU')}")

TensorFlow использует GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
dataset_path = '/content/drive/MyDrive/lego/static/all_images'

In [ ]:

IMAGE_SIZE = 512 # Размер изображения

# Сбор уникальных классов и подсчет общего количества изображений
class_counts = defaultdict(int)
total_images = 0
image_data = []
labels = []

# Функция обработки изображений
def preprocess_image(image_path, image_size=512, center_fraction=0.4, diff_threshold=15,
                     edge_exclusion_top=0.2, edge_exclusion_bottom=0.2,
                     edge_exclusion_left=0.2, edge_exclusion_right=0.3):
    """
    Обработка изображения: обнуление фона, исключение краевых областей с разной настройкой для каждой стороны.
    """
    # Загрузка изображения
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Исключение краевых областей
    h, w = gray_img.shape
    top = int(h * edge_exclusion_top)
    bottom = int(h * (1 - edge_exclusion_bottom))
    left = int(w * edge_exclusion_left)
    right = int(w * (1 - edge_exclusion_right))

    # Создаем маску для центральной области
    mask_edges = np.zeros_like(gray_img, dtype=np.uint8)
    mask_edges[top:bottom, left:right] = 255  # Центральная часть остается

    # Применяем маску краевых областей
    gray_img = cv2.bitwise_and(gray_img, gray_img, mask=mask_edges)

    # Определяем фон по центральной области
    center_x_start = int(w * (0.5 - center_fraction / 2))
    center_x_end = int(w * (0.5 + center_fraction / 2))
    center_y_start = int(h * (0.5 - center_fraction / 2))
    center_y_end = int(h * (0.5 + center_fraction / 2))
    central_region = gray_img[center_y_start:center_y_end, center_x_start:center_x_end]
    background_value = np.median(central_region)

    # Заменяем пиксели за пределами центральной области на значение фона
    gray_img[mask_edges == 0] = int(background_value)

    # Создаем маску объекта
    diff = np.abs(gray_img - background_value)
    mask = (diff > diff_threshold).astype(np.uint8)

    # Дополнительное улучшение выделения деталей
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Убираем шум
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)  # Расширяем объект

    # Применяем маску к изображению
    processed_img = cv2.bitwise_and(gray_img, gray_img, mask=mask)

    # Изменяем размер для нейросети
    resized_img = cv2.resize(processed_img, (image_size, image_size))

    return resized_img

# Перебор всех файлов в директории
for filename in os.listdir(dataset_path):
    if os.path.isfile(os.path.join(dataset_path, filename)) and ('-' in filename):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Извлечение номера класса из названия файла
            parts = filename.split('-')
            if len(parts) == 2 and parts[0].isdigit():
                class_name = parts[0]  # Номер класса
                class_counts[class_name] += 1
                total_images += 1

                # Путь к изображению
                img_path = os.path.join(dataset_path, filename)

                # Обработка изображения
                processed_image = preprocess_image(img_path, image_size=IMAGE_SIZE)

                # Добавление обработанного изображения и метки класса
                image_data.append(processed_image)
                labels.append(int(class_name))  # Преобразуем метку класса в целое число

# Конвертируем данные в массивы numpy
image_data = np.array(image_data)
labels = np.array(labels)

# Преобразуем метки классов в последовательность от 0 до N-1
unique_classes = sorted(set(labels))  # Сортируем уникальные классы
class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}  # Словарь для сопоставления классов с индексами

# Преобразуем метки в новые индексы
labels = np.array([class_to_index[cls] for cls in labels])

# Нормализация данных (пиксели в диапазоне [0, 1])
image_data = image_data.astype('float32') / 255.0

# Преобразование меток в one-hot encoding
labels = to_categorical(labels, num_classes=len(unique_classes))

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

In [ ]:
import random
# Выбираем случайное изображение
random_idx = random.randint(0, len(image_data) - 1)
sample_image = image_data[random_idx]
if len(sample_image.shape) == 2:  # Проверяем, что это 2D изображение (чёрно-белое)
    plt.imshow(sample_image, cmap='gray')
elif len(sample_image.shape) == 3 and sample_image.shape[-1] == 1:  # Если добавлен канал, убираем его
    plt.imshow(sample_image[:, :, 0], cmap='gray')
else:
    raise ValueError("Неожиданный формат изображения для отображения.")

# Настраиваем заголовок и показываем изображение
plt.title("Пример обработанного изображения")
plt.axis('off')
plt.show()


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_counts), activation='softmax'))

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Обучение модели
epochs = 30  # Увеличиваем количество эпох для лучшего обучения
batch_size = 16  # Уменьшаем размер батча, чтобы учесть увеличение изображения
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Оценка модели на тестовой выборке
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n📊 Точность модели на тестовых данных: {accuracy:.2f}")
print(f"📊 Потери (loss) на тестовых данных: {loss:.4f}")

In [ ]:
# Запрос у пользователя, какое изображение он хочет тестировать
test_image_name = input("Введите имя изображения для тестирования (например, 84-1): ").strip()

# Ищем все изображения с указанным классом
matching_images = [f for f in os.listdir(dataset_path) if f.startswith(test_image_name) and f.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [ ]:
if not matching_images:
    print(f"Ошибка: изображения для класса {test_image_name} не найдены.")

In [ ]:
for test_image_name in matching_images:
    test_image_path = os.path.join(dataset_path, test_image_name)

    # Извлекаем реальный класс из имени файла
    real_class = test_image_name.split('-')[0]  # Извлекаем номер класса из имени файла

    # Обработка тестового изображения (используем ту же функцию)
    processed_test_img = preprocess_image(
        test_image_path,
        image_size=IMAGE_SIZE,
        center_fraction=0.4,
        diff_threshold=15,
        edge_exclusion_top=0.2,
        edge_exclusion_bottom=0.2,
        edge_exclusion_left=0.2,
        edge_exclusion_right=0.3
    )

    # Преобразуем обработанное изображение в 4D массив для модели
    test_img_array = processed_test_img.reshape(1, IMAGE_SIZE, IMAGE_SIZE, 1)  # Добавляем размерность для партии

    # Нормализуем изображение
    test_img_array = test_img_array.astype('float32') / 255.0

    # Прогнозируем класс изображения
    prediction = model.predict(test_img_array)
    predicted_class_idx = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction) * 100
    predicted_class = unique_classes[predicted_class_idx]  # Преобразуем индекс обратно в реальный класс

    # Выводим результаты
    print(f"📷 Реальный класс: {real_class}")
    print(f"🤖 Предсказанный класс: {predicted_class}")
    print(f"🎯 Уверенность в предсказании: {confidence:.2f}%")

    # Выводим изображение и его классы
    plt.imshow(processed_test_img, cmap='gray')
    plt.title(f"Real: {real_class} | Predicted: {predicted_class} | Confidence: {confidence:.2f}%")
    plt.axis('off')
    plt.show()


In [ ]:
# Для подробного тестирования на других изображениях:
print(f"Общее количество уникальных классов: {len(class_counts)}")
print(f"Общее количество изображений: {total_images}")
print("Уникальные классы:")
for cls in sorted(class_counts):
    print(cls)

In [ ]:
# Настройка размера изображений
IMAGE_SIZE = 512  # Размер изображения

# Сбор уникальных классов и подсчет общего количества изображений
class_counts = defaultdict(int)
total_images = 0
image_data = []
labels = []

# Перебор всех файлов в директории
for filename in os.listdir(dataset_path):
    if os.path.isfile(os.path.join(dataset_path, filename)) and ('-' in filename):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Извлечение номера класса из названия файла
            parts = filename.split('-')
            if len(parts) == 2 and parts[0].isdigit():
                class_name = parts[0]  # Номер класса
                class_counts[class_name] += 1
                total_images += 1

                # Загрузка изображения
                img_path = os.path.join(dataset_path, filename)
                img = Image.open(img_path).convert('L')  # Конвертация в черно-белое (оттенки серого)
                img = img.resize((IMAGE_SIZE, IMAGE_SIZE))  # Изменение размера изображения

                # Преобразуем изображение в массив numpy
                img_array = np.array(img)
                image_data.append(img_array)
                labels.append(int(class_name))  # Преобразуем метку класса в целое число

# Конвертируем данные в массивы numpy
image_data = np.array(image_data)
labels = np.array(labels)

# Преобразуем метки классов в последовательность от 0 до N-1
unique_classes = sorted(set(labels))  # Сортируем уникальные классы
class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}  # Словарь для сопоставления классов с индексами

# Преобразуем метки в новые индексы
labels = np.array([class_to_index[cls] for cls in labels])

# Нормализация данных (пиксели в диапазоне [0, 1])
image_data = image_data.astype('float32') / 255.0

# Преобразование меток в one-hot encoding
labels = to_categorical(labels, num_classes=len(class_counts))

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)